# Accessing API to Download Data
# 00_get_data

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
|    |  |    |  | 
|    | |   | |

### Content

* [Introduction](#introduction)

In [1]:
import requests
import time
import json

### API Sandbox Credentials

In [ ]:
# Public Credentials for sandbox

# partially adjudicated: claim, claimResponse
# fully adjudicated: patient, coverage, explanationOfBenefit

# Extra-small REACH ACO (110 synthetic enrollees) - Partially Adjudicated
# client_id = '7e57394f-eddb-46c7-a87b-a23f14ded95d'
# client_secret = '3ab22e7faaf69fa2d572831ffc1db12252c6d569d3e1b54aecf56e075ba054c20fee83b2e013c9c3'

# Large REACH ACO (11,000 synthetic enrollees) - Paritally Adjudicated XXXUse this
#client_id = '2121efbd-98d2-4323-84db-974c8864abc7'
#client_secret = 'ee1b0609f024a758bf1770ec16f809330d2ba8bb4e9004a7868c0258accfd69ced5b6448188abb7b'

# Large advanced model entity (10,000 synthetic enrollees) - Adjudicated
# client_id = '0a0c75f0-da95-4198-9c0f-666b41e21017'
# client_secret = 'c637024fa21adda5a756a2753cf7eb9bd62292e7897fb965a5c7aeeed23e1728ddc9ec6863f09f15'

# Extra-large model entity (30,000 synthetic enrollees) - Adjudicated XXXUse this
client_id = 'aa2d6b93-bbe7-4d1b-8cc5-9a5172fae3a6'
client_secret = '97755772b3fb7b3fa2f58c5c3aaaffbc7e346639ff8da371a81adf79889c8fbd4c40398cd39d211d'

### Functions to Download Data from API

In [3]:
def get_access_token(client_id, client_secret):
    """
    Obtain an access token using client credentials.
    """
    url = 'https://sandbox.bcda.cms.gov/auth/token'
    headers = {
        'accept': 'application/json'
    }
    response = requests.post(url, headers=headers, auth=(client_id, client_secret))
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("Access token obtained successfully.")
        return access_token
    else:
        print(f"Failed to get access token: {response.status_code}, {response.text}")
        return None

In [4]:
def start_job(access_token, dataset):
    """
    Start a data export job.
    """
    if dataset!='':
        url = 'https://sandbox.bcda.cms.gov/api/v2/Group/all/$export?_type=' + dataset
        print(url)
    else:
        url = 'https://sandbox.bcda.cms.gov/api/v2/Group/all/$export'
    headers = {
        'accept': 'application/fhir+json',
        'Prefer': 'respond-async',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 202:
        job_url = response.headers.get('Content-Location')
        print(f"Job started successfully. Job URL: {job_url}")
        return job_url
    else:
        print(f"Failed to start job: {response.status_code}, {response.text}")
        return None

In [5]:
def check_job_status(job_url, access_token):
    """
    Check the status of the data export job until it is complete.
    """
    headers = {
        'accept': 'application/fhir+json',
        'Authorization': f'Bearer {access_token}'
    }
    while True:
        response = requests.get(job_url, headers=headers)
        if response.status_code == 202:
            # Job is still processing
            print("Job is still processing... Checking again in 10 seconds.")
            time.sleep(10)  # Wait before checking again
        elif response.status_code == 200:
            # Job is done
            print("Job completed successfully.")
            return response.json()
        else:
            print(f"Failed to check job status: {response.status_code}, {response.text}")
            return None

In [6]:
def download_data_files(output_list, access_token):
    """
    Download the data files from the job output.
    """
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept-Encoding': 'gzip'
    }
    for item in output_list:
        file_type = item.get('type')
        url = item.get('url')
        print(f"Downloading {file_type} data from {url}...")
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            filename = f"../data/raw/{file_type}.ndjson"
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Saved {filename}")
        else:
            print(f"Failed to download {file_type}: {response.status_code}, {response.text}")

In [7]:
def main(dataset=''):
    access_token = get_access_token(client_id, client_secret)
    if not access_token:
        return
    job_url = start_job(access_token, dataset)
    if not job_url:
        return
    job_result = check_job_status(job_url, access_token)
    if not job_result:
        return
    output = job_result.get('output')
    if not output:
        print("No output data available.")
        return
    download_data_files(output, access_token)

Have two options to download either each dataset individually or all datasets available at that access point

In [12]:
# Use to download a specific dataset
if __name__ == "__main__":
    main(dataset = 'Patient')

Access token obtained successfully.
https://sandbox.bcda.cms.gov/api/v2/Group/all/$export?_type=Patient
Job started successfully. Job URL: https://sandbox.bcda.cms.gov/api/v2/jobs/80606
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job completed successfully.
Saved ../data/raw/Patient.ndjson
Saved ../data/raw/Patient.ndjson


In [10]:
# Downloads all datasets
# if __name__ == "__main__":
#     main(dataset = '')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cff59b50-5bfa-41ed-b840-afb0c6e184df' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>